# Azure ML - Sample Batch Prediction Pipeline
- Parallel run step leveraged

In [1]:
import azureml.core
from azureml.core import Workspace
import os, shutil
import pandas as pd
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData, PipelineEndpoint
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.core.experiment import Experiment

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.38.0 to work with mlopsdev


In [2]:
import os, shutil
folder_name = 'batch-inferencing'
script_folder = os.path.join(os.getcwd(), folder_name)
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/EmailClass/email_parallelRun_classificationbatch_pipeline/batch-inferencing


In [3]:
#Get default datastore
default_ds = ws.get_default_datastore()

### In order to batch inference the data and leverage the parallel run step, we want to break up the dataset.
The code below will generate many files - and then register them in the default datastore under the spam-data-inferencing folder

In [4]:
#spam-data-inferencing

data = pd.read_csv('./datasets/spaminfernce.csv')
data = data[['text']]

# Create a folder
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# # Save each sample as a separate file
# print("Saving files...")
# for i in range(100):
#     sample[i].tofile(os.path.join(batch_folder, str(i+1) + '.csv'), sep=",")
# print("files saved!")

k = 100
size = 1

for i in range(k):
    df = data[size*i:size*(i+1)]
    df.to_csv(f'./batch-data/{i+1}.csv', index=False)
    
print('files created')

Folder created!
files created


In [5]:
# Upload the files to the default datastore
print("Uploading files to datastore...")
default_ds = ws.get_default_datastore()
default_ds.upload(src_dir="batch-data", target_path="spam-data-inferencing", overwrite=True, show_progress=True)

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'spam-data-inferencing/*.csv'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='spam-batch-data-inference',
                                             description='inference batch data',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading files to datastore...
Uploading an estimated of 100 files
Uploading batch-data/1.csv
Uploaded batch-data/1.csv, 1 files out of an estimated total of 100
Uploading batch-data/100.csv
Uploaded batch-data/100.csv, 2 files out of an estimated total of 100
Uploading batch-data/12.csv
Uploaded batch-data/12.csv, 3 files out of an estimated total of 100
Uploading batch-data/13.csv
Uploaded batch-data/13.csv, 4 files out of an estimated total of 100
Uploading batch-data/15.csv
Uploaded batch-data/15.csv, 5 files out of an estimated total of 100
Uploading batch-data/16.csv
Uploaded batch-data/16.csv, 6 files out of an estimated total of 100
Uploading batch-data/19.csv
Uploaded batch-data/19.csv, 7 files out of an estimated total of 100
Uploading batch-data/21.csv
Uploaded batch-data/21.csv, 8 files out of an estimated total of 100
Uploading batch-data/24.csv
Uploaded batch-data/24.csv, 9 files out of an estimated total of 100
Uploading batch-data/27.csv
Uploaded batch-data/27.csv, 10 

Create Cluster

In [6]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException

compute_name =  "email-cluster"
print(compute_name)

# checks to see if compute target already exists in workspace, else create it
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D13_V2",
                                                   min_nodes=0, 
                                                   max_nodes=5)

    compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=120)

email-cluster
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
%%writefile $script_folder/email_classification_inference.yml
name: email_classification_inference
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Writing /mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/EmailClass/email_parallelRun_classificationbatch_pipeline/batch-inferencing/email_classification_inference.yml


In [8]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create an Environment for the experiment
batch_env = Environment.from_conda_specification("email_classification_inference", script_folder + "/email_classification_inference.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Configuration ready.


# Define Pipeline Parameters

PipelineParameter objects serve as variable inputs to an Azure ML pipeline and can be specified at runtime. Below we specify a pipeline parameter object model_name which will be used to reference the locally trained model that was uploaded and registered within the Azure ML workspace. Multiple pipeline parameters can be created and used. Included here are multiple sample pipeline parameters (get_data_param_*) to highlight how parameters can be passed into and consumed by various pipeline steps.

In [9]:
model_name = PipelineParameter(name='model_name', default_value='email_classifier')

# Define Pipeline Steps

The pipeline below consists of steps to gather and register data from a remote source, a scoring step where the registered model is used to make predictions on loaded, and a data publish step where scored data can be exported to a remote data source. All of the PythonScriptSteps have a corresponding *.py file which is referenced in the step arguments. Also, any PipelineParameters defined above can be passed to and consumed within these steps.


In [10]:
import os, shutil
folder_name = 'batch-inferencing'
script_folder = os.path.join(os.getcwd(), folder_name)
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/EmailClass/email_parallelRun_classificationbatch_pipeline/batch-inferencing


In [11]:
%%writefile $script_folder/batch_inferencing_data.py

import os
import numpy as np
from azureml.core import Model
import joblib
import pandas as pd

def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    print('****loaded model**********')
    model_path = Model.get_model_path('email_classifier')
    model = joblib.load(model_path)


def run(mini_batch):
    # This runs for each batch
    print(f'run method start: {__file__}, run({mini_batch})')
    resultList = []
    print('type of mini batch')
    print(str(type(mini_batch)))
    # process each file in the batch
    for f in mini_batch:
        print('****working on mini_batch**********')
        print(f)
        #open text file in read mode
        text_file = open(f, "r")
        data = text_file.read()
        text_file.close()
        result = model.predict([data])
        print(data)
        resultList.append("{}: {}".format(os.path.basename(f), result[0]))
    return resultList

Writing /mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/EmailClass/email_parallelRun_classificationbatch_pipeline/batch-inferencing/batch_inferencing_data.py


You're going to use a pipeline to run the batch prediction script, generate predictions from the input data, and save the results as a text file in the output folder. To do this, you can use a **ParallelRunStep**, which enables the batch data to be processed in parallel and the results collated in a single output file named *parallel_run_step.txt*.

In [12]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig
from azureml.core.runconfig import DockerConfiguration

output_dir = OutputFileDatasetConfig(name='inferences')

parallel_run_config = ParallelRunConfig(
    source_directory=script_folder,
    entry_script="batch_inferencing_data.py",
    mini_batch_size="50",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=compute_target,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-score-diabetes',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('email_batch')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Steps defined


In [13]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, '02-email-classifcation-batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step batch-score-diabetes [8fc2b277][bee8c409-9ffd-4b24-a463-aad5e89d8c21], (This step will run and generate new outputs)
Submitted PipelineRun 69d940b3-cf03-411e-863f-b2393ee171c2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/69d940b3-cf03-411e-863f-b2393ee171c2?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRunId: 69d940b3-cf03-411e-863f-b2393ee171c2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/69d940b3-cf03-411e-863f-b2393ee171c2?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: NotStarted
PipelineRun Status: Running




PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': '69d940b3-cf03-411e-863f-b2393ee171c2', 'status': 'Completed', 'startTimeUtc': '2022-02-09T14:28:46.525152Z', 'endTimeUtc': '2022-02-09T14:45:34.

Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


'Finished'

## Retrieve results

In [14]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
try:
    shutil.rmtree('diabetes-results', ignore_errors=True)
except:
    print('keep going dude')

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='diabetes-results')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('diabetes-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

AttributeError: 'Run' object has no attribute 'get_output_data'

In [ ]:
# pipeline_id = pipeline.Id
# experiment_name = 'silly_scheduled_email'
# recurrence = ScheduleRecurrence(frequency="Minute", interval=5)
# recurring_schedule = Schedule.create(ws, name="MyRecurringSchedule", 
#                             description="Based on time",
#                             pipeline_id=pipeline_id, 
#                             experiment_name=experiment_name, 
#                             recurrence=recurrence)